In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

train_ud = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_per_relation_no_relation.csv')
test_ud = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_per_relation_no_relation.csv')

train_ucca = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ucca_paths_per_relation_no_relation.csv')
test_ucca = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ucca_paths_per_relation_no_relation.csv')

In [3]:
train_ud = train_ud.rename(columns={'path' : 'ud_path'})
test_ud = test_ud.rename(columns={'path' : 'ud_path'})

train_ucca = train_ucca.rename(columns={'path' : 'ucca_path'})
test_ucca = test_ucca.rename(columns={'path' : 'ucca_path'})

In [4]:
test = pandas.merge(test_ud, test_ucca[['id', 'ucca_path']], on='id').dropna()

In [5]:
relation_to_paths_ud = train_ud.groupby(['relation'])['ud_path'].unique().to_dict()
relation_to_paths_ucca = train_ucca.groupby(['relation'])['ucca_path'].unique().to_dict()

In [6]:
def label_sentence_recall(test_row, relation):
    ud_path = test_row['ud_path']
    ucca_path = test_row['ucca_path']
    true_relation = test_row['relation']
    
    if ud_path in relation_to_paths_ud[relation] or ucca_path in relation_to_paths_ucca[relation]:
        if true_relation == relation:
            return 'TP'
        
        # the next two lines are artificial and should be removed
        # they are designed to remove the affect of all 'no_relation' sentences
        #elif true_relation == 'no_relation':
        #    return 'N'
        
        
        else:
            return 'FP'
    else:
        return 'N'

In [7]:
for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths_ud.keys()):
    test[relation] = test.apply(lambda row : label_sentence_recall(row, relation), axis=1)    

In [8]:
stats = []
overall_tp = 0
overall_fp = 0

for relation in filter(lambda relation: relation != 'no_relation', relation_to_paths_ud.keys()):

    counts = test[relation].value_counts().to_dict()
    num_examples = test[test['relation']==relation].shape[0]
        
    tp = counts.get('TP', 0)
    fp = counts.get('FP', 0)
    precision = tp / (tp+fp)
    
    #print('{}: total ={}, tp = {}, fp = {}'.format(relation, num_examples, tp, fp))

    stats.append( ( '{} ({})'.format(relation,num_examples), round(precision,4)) )
    
    overall_tp += tp
    overall_fp += fp

    
overall_precision = overall_tp / (overall_tp+overall_fp)

    
#matched = sum(test_total_by_r_and_p[test_total_by_r_and_p['total-for-relation']>0]['total'].to_list())
#matched_lax = sum(test_total_by_r_and_p[test_total_by_r_and_p['total-across-relations']>0]['total'].to_list())
#total = sum(test_total_by_r_and_p['total'].to_list())

stats.append( ('overall', round(overall_precision,4)) )
stats = sorted(stats, key=lambda x: x[1], reverse=True)

In [9]:
import plotly.graph_objs as go

overall_index = next(i for (i, (relation, score)) in enumerate(stats) if relation == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'


trace1 = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Precision',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace1]
layout = go.Layout(
    title='UD + UCCA Path - Precision',
    xaxis_title="Relation",
    yaxis_title="Precision",        
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)